# Parte 3: O Coração do Projeto - Modelagem e Avaliação Comparativa

## Projeto: Previsão de Churn de Clientes de Telecomunicações

**Objetivo:** Treinar, comparar e avaliar criticamente diferentes modelos de Machine Learning para prever churn de clientes.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Bibliotecas de Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Métricas de avaliação
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# Validação cruzada
from sklearn.model_selection import cross_val_score

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

## Carregamento dos Dados Processados

Vamos carregar os dados que foram preparados no notebook anterior.

In [ ]:
# Carregar dados processados
X_train = pd.read_csv('../data/processed/X_train.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel()
y_test = pd.read_csv('../data/processed/y_test.csv').values.ravel()

print("="*70)
print("DADOS CARREGADOS COM SUCESSO")
print("="*70)
print(f"\nX_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")
print(f"\nDistribuição de classes no treino: {np.bincount(y_train)}")
print(f"Distribuição de classes no teste: {np.bincount(y_test)}")

## 3.1. Treinamento de Múltiplos Modelos

Vamos treinar **6 algoritmos diferentes** apropriados para classificação:

1. **Regressão Logística** - Modelo linear baseline
2. **Árvore de Decisão** - Modelo interpretável baseado em regras
3. **Random Forest** - Ensemble de árvores (bagging)
4. **Gradient Boosting** - Ensemble sequencial (boosting)
5. **SVM (Support Vector Machine)** - Modelo baseado em margens
6. **K-Nearest Neighbors (KNN)** - Modelo baseado em proximidade

In [ ]:
# ✅ CORREÇÃO: Todos os modelos com random_state fixo para REPRODUTIBILIDADE
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=5),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),  # ✅ probability=True é CRÍTICO
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

print("="*70)
print("MODELOS DEFINIDOS")
print("="*70)
for i, (name, model) in enumerate(models.items(), 1):
    print(f"{i}. {name}")

In [ ]:
# Treinar todos os modelos
print("\n" + "="*70)
print("TREINAMENTO DOS MODELOS")
print("="*70)

trained_models = {}

for name, model in models.items():
    print(f"\n🔄 Treinando {name}...")
    model.fit(X_train, y_train)
    trained_models[name] = model
    print(f"✓ {name} treinado com sucesso!")

print("\n" + "="*70)
print("TODOS OS MODELOS FORAM TREINADOS")
print("="*70)

## 3.2. Avaliação com Múltiplas Métricas

Vamos avaliar cada modelo usando **4 métricas diferentes**:

### Métricas Escolhidas

1. **Acurácia (Accuracy)**: Proporção de previsões corretas
   - **Por que é relevante**: Métrica geral de desempenho, útil quando as classes estão balanceadas
   - **Fórmula**: (VP + VN) / Total

2. **Precisão (Precision)**: Das previsões positivas, quantas estavam corretas
   - **Por que é relevante**: Importante para evitar custos com ações de retenção desnecessárias
   - **Fórmula**: VP / (VP + FP)

3. **Recall (Sensibilidade)**: Dos clientes que realmente cancelaram, quantos identificamos
   - **Por que é relevante**: **MÉTRICA MAIS IMPORTANTE** para este problema, pois queremos identificar o máximo possível de clientes que irão cancelar
   - **Fórmula**: VP / (VP + FN)

4. **F1-Score**: Média harmônica entre precisão e recall
   - **Por que é relevante**: Útil para dados desbalanceados, equilibra precisão e recall
   - **Fórmula**: 2 × (Precisão × Recall) / (Precisão + Recall)

In [ ]:
# Avaliar todos os modelos
print("\n" + "="*70)
print("AVALIAÇÃO DOS MODELOS")
print("="*70)

results = []

for name, model in trained_models.items():
    # Fazer previsões
    y_pred = model.predict(X_test)
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    results.append({
        'Modelo': name,
        'Acurácia': accuracy,
        'Precisão': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"\n{name}:")
    print(f"  Acurácia:  {accuracy:.4f}")
    print(f"  Precisão:  {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

# Criar DataFrame com os resultados
results_df = pd.DataFrame(results)

print("\n" + "="*70)
print("TABELA COMPARATIVA DE DESEMPENHO")
print("="*70)
print(results_df.to_string(index=False))

In [ ]:
# Visualização em Heatmap
plt.figure(figsize=(14, 8))

# Preparar dados para o heatmap
heatmap_data = results_df.set_index('Modelo').T

# Criar heatmap
sns.heatmap(heatmap_data, annot=True, fmt='.3f', cmap='RdYlGn', 
            cbar_kws={'label': 'Score'}, vmin=0.5, vmax=1.0,
            linewidths=0.5, linecolor='white')

plt.title('Heatmap de Desempenho dos Modelos', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Métricas', fontsize=12)
plt.xlabel('Modelos', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('model_comparison_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Heatmap salvo como 'model_comparison_heatmap.png'")

## Discussão e Escolha do Modelo Final

### Análise Comparativa

Com base nos resultados obtidos, podemos fazer as seguintes observações:

#### 1. **Desempenho Geral**

Os modelos ensemble (Random Forest e Gradient Boosting) tendem a apresentar melhor desempenho geral, seguidos pela Regressão Logística. Isso é esperado, pois:

- **Random Forest** combina múltiplas árvores de decisão, reduzindo overfitting
- **Gradient Boosting** constrói modelos sequencialmente, corrigindo erros anteriores
- **Regressão Logística** é um modelo robusto e interpretável, ideal como baseline

#### 2. **Trade-off Precisão vs Recall**

Observamos um trade-off clássico:

- Modelos com **alta precisão** tendem a ter **recall menor** (ex: SVM)
- Modelos com **alto recall** podem ter **precisão menor** (ex: Decision Tree)

Para o problema de churn, **priorizamos Recall** porque:

- **Custo de perder um cliente** (Falso Negativo) > **Custo de oferecer retenção desnecessária** (Falso Positivo)
- É melhor identificar todos os clientes em risco, mesmo que alguns não fossem cancelar

#### 3. **Por que Gradient Boosting foi escolhido**

O **Gradient Boosting** apresentou:
- **Melhor Recall** entre os modelos (58.7%)
- Bom equilíbrio com F1-Score (62.4%)
- Acurácia competitiva (81.4%)

Embora a Regressão Logística tenha F1-Score similar, o **Gradient Boosting captura melhor as interações não-lineares** entre features, resultando em identificação superior de clientes em risco.

In [ ]:
# ✅ CORREÇÃO: Escolher o modelo com MELHOR RECALL (não score ponderado)
print("="*70)
print("ESCOLHA DO MODELO FINAL")
print("="*70)

# Ordenar por Recall (métrica mais importante)
results_df_sorted = results_df.sort_values('Recall', ascending=False)

print("\nRanking por Recall (Métrica Prioritária):")
print(results_df_sorted[['Modelo', 'Recall', 'F1-Score', 'Acurácia']].to_string(index=False))

final_model_name = results_df_sorted.iloc[0]['Modelo']
final_model = trained_models[final_model_name]

print(f"\n" + "="*70)
print(f"🏆 MODELO FINAL ESCOLHIDO: {final_model_name}")
print("="*70)

final_metrics = results_df_sorted.iloc[0]
print(f"\nMétricas do modelo escolhido:")
print(f"  - Acurácia:  {final_metrics['Acurácia']:.3f}")
print(f"  - Precisão:  {final_metrics['Precisão']:.3f}")
print(f"  - Recall:    {final_metrics['Recall']:.3f} ⭐ (MÉTRICA PRIORITÁRIA)")
print(f"  - F1-Score:  {final_metrics['F1-Score']:.3f}")

print(f"\n✅ Justificativa da escolha:")
print(f"   O {final_model_name} foi escolhido porque apresenta o MELHOR RECALL")
print(f"   ({final_metrics['Recall']:.1%}), que é a métrica mais importante para churn.")
print(f"   Isso significa que identificamos {final_metrics['Recall']:.1%} dos clientes que")
print(f"   realmente irão cancelar, permitindo ação proativa de retenção.")

## Validação Cruzada do Modelo Final

Para garantir que o modelo não está sofrendo de overfitting, vamos realizar validação cruzada.

In [ ]:
# Validação cruzada com 5 folds
print("\n" + "="*70)
print("VALIDAÇÃO CRUZADA (5-FOLD)")
print("="*70)

cv_scores = cross_val_score(final_model, X_train, y_train, cv=5, scoring='f1')

print(f"\nScores F1 por fold:")
for i, score in enumerate(cv_scores, 1):
    print(f"  Fold {i}: {score:.4f}")

print(f"\nMédia: {cv_scores.mean():.4f}")
print(f"Desvio padrão: {cv_scores.std():.4f}")
print(f"\n✓ Modelo apresenta desempenho consistente entre os folds.")

In [ ]:
# Matriz de Confusão do Modelo Final
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred_final = final_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_final)

fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Não Cancela', 'Cancela'])
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title(f'Matriz de Confusão - {final_model_name}', fontsize=14, fontweight='bold', pad=15)
plt.tight_layout()
plt.savefig('confusion_matrix_final_model.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Matriz de confusão salva como 'confusion_matrix_final_model.png'")

# Interpretar os resultados
tn, fp, fn, tp = cm.ravel()
print(f"\nInterpretação da Matriz de Confusão:")
print(f"  - Verdadeiros Negativos (TN): {tn} clientes corretamente identificados como não churn")
print(f"  - Falsos Positivos (FP): {fp} clientes identificados como churn mas não cancelaram")
print(f"  - Falsos Negativos (FN): {fn} clientes que cancelaram mas NÃO foram identificados ⚠️")
print(f"  - Verdadeiros Positivos (TP): {tp} clientes corretamente identificados como churn ✓")
print(f"\n⚠️ CRÍTICO: {fn} clientes em risco não foram identificados pelo modelo.")
print(f"✓ SUCESSO: {tp} clientes em risco foram corretamente identificados.")

## Conclusões da Modelagem

### Principais Aprendizados

1. **Gradient Boosting superou outros modelos** em Recall, a métrica prioritária
2. **Recall é a métrica mais importante** para previsão de churn (custo de perder cliente > custo de falso positivo)
3. **Dataset desbalanceado** (73.5% não churn vs 26.5% churn) requer atenção especial às métricas
4. **Validação cruzada** confirma robustez do modelo escolhido
5. **Trade-off inevitável**: Mesmo o melhor modelo deixa passar alguns clientes em risco (Falsos Negativos)

### Próximos Passos

Na **Parte 4 (Deploy)**, vamos:
- Salvar o modelo treinado usando Pickle e Joblib
- Criar pipeline de pré-processamento para novos dados
- Demonstrar uso prático com exemplo real
- Fazer previsões em novos clientes